<a href="https://colab.research.google.com/github/Codekster/RAG-practice/blob/main/RAGTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community langchainhub chromadb langchain langchain-openai


In [ ]:
!pip install langchain_google_genai

In [36]:
from google.colab import userdata
import os

# Assuming your Gemini API key is stored as 'GOOGLE_API_KEY' in Colab secrets
gemini_api_key = userdata.get('Googlekey')
os.environ['GOOGLE_API_KEY'] = gemini_api_key

print("Gemini API key has been set as an environment variable.")

Gemini API key has been set as an environment variable.


In [37]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

page_url = ["https://en.wikipedia.org/wiki/World_War_II"]

loader = WebBaseLoader(web_paths=page_url)
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

print(f"Fetched {len(docs)} documents")

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  4.86it/s]


Fetched 1 documents


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load example document


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=3000,
    chunk_overlap=300,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.split_documents(docs)
print(texts[0])
print(texts[1])


In [39]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
import os

# Ensure GOOGLE_API_KEY is set from the previous cell
if 'GOOGLE_API_KEY' not in os.environ:
    print("GOOGLE_API_KEY not set. Please run the cell to set the environment variable.")
else:
    vectorstore = Chroma.from_documents(documents=texts, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
    print("Embeddings created and stored in Chroma.")

Embeddings created and stored in Chroma.


In [41]:
retriever=vectorstore.as_retriever()

In [42]:
from langchain import hub
prompt=hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [43]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os

# Ensure GOOGLE_API_KEY is set
if 'GOOGLE_API_KEY' not in os.environ:
    print("GOOGLE_API_KEY not set. Please run the cell to set the environment variable.")
else:
    llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", temperature=0.5)
    print("Gemini LLM instance created.")

Gemini LLM instance created.


In [44]:
def promptJoin(prompt):
  return "\n".join(doc.page_content for doc in prompt )

In [32]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
rag_chain=({"context":retriever | promptJoin ,"question":RunnablePassthrough()}
| prompt
| llm
| StrOutputParser()
)

In [49]:
rag_chain.invoke("When did the war start?")

'The provided context is a table of contents and further reading for a Wikipedia article on World War II. While it mentions "Start and end dates" as a section, the actual dates are not present in the provided text. Therefore, I don\'t know the answer.'